In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score
from collections import Counter
from math import log2

In [ ]:
# Function to compute Gini Impurity
def gini_impurity(y):
    # Get the proportion of each class
    proportions = np.bincount(y) / len(y)
    # Calculate Gini
    gini = 1 - np.sum([p**2 for p in proportions if p > 0])
    return gini

In [ ]:
# Function to compute Entropy
def entropy(y):
    # Get the proportion of each class
    proportions = np.bincount(y) / len(y)
    # Calculate entropy
    ent = -np.sum([p * log2(p) for p in proportions if p > 0])
    return ent

In [ ]:
dataset = "/content/drive/MyDrive/Project II (Sem VII)/Liver Patient Dataset (LPD)_train.csv"

In [ ]:
data = pd.read_csv(dataset, encoding='ISO-8859-1')

In [ ]:
data

Age of the patient Gender of the patient  Total Bilirubin  \
0                    65.0                Female              0.7   
1                    62.0                  Male             10.9   
2                    62.0                  Male              7.3   
3                    58.0                  Male              1.0   
4                    72.0                  Male              3.9   
...                   ...                   ...              ...   
30686                50.0                  Male              2.2   
30687                55.0                  Male              2.9   
30688                54.0                  Male              6.8   
30689                48.0                Female              1.9   
30690                30.0                  Male              3.1   

       Direct Bilirubin   Alkphos Alkaline Phosphotase  \
0                   0.1                          187.0   
1                   5.5                          699.0   
2                   4.1                          490.0   
3                   0.4                          182.0   
4                   2.0                          195.0   
...                 ...                            ...   
30686               1.0                          610.0   
30687               1.3                          482.0   
30688               3.0                          542.0   
30689               1.0                          231.0   
30690               1.6                          253.0   

        Sgpt Alamine Aminotransferase  Sgot Aspartate Aminotransferase  \
0                                16.0                             18.0   
1                                64.0                            100.0   
2                                60.0                             68.0   
3                                14.0                             20.0   
4                                27.0                             59.0   
...                               ...                              ...   
30686                            17.0                             28.0   
30687                            22.0                             34.0   
30688                           116.0                             66.0   
30689                            16.0                             55.0   
30690                            80.0                            406.0   

       Total Protiens   ALB Albumin  A/G Ratio Albumin and Globulin Ratio  \
0                 6.8           3.3                                  0.90   
1                 7.5           3.2                                  0.74   
2                 7.0           3.3                                  0.89   
3                 6.8           3.4                                  1.00   
4                 7.3           2.4                                  0.40   
...               ...           ...                                   ...   
30686             7.3           2.6                                  0.55   
30687             7.0           2.4                                  0.50   
30688             6.4           3.1                                  0.90   
30689             4.3           1.6                                  0.60   
30690             6.8           3.9                                  1.30   

       Result  
0           1  
1           1  
2           1  
3           1  
4           1  
...       ...  
30686       1  
30687       1  
30688       1  
30689       1  
30690       1  

[30691 rows x 11 columns]

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30691 entries, 0 to 30690
Data columns (total 11 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Age of the patient                    30689 non-null  float64
 1   Gender of the patient                 29789 non-null  object 
 2   Total Bilirubin                       30043 non-null  float64
 3   Direct Bilirubin                      30130 non-null  float64
 4    Alkphos Alkaline Phosphotase         29895 non-null  float64
 5    Sgpt Alamine Aminotransferase        30153 non-null  float64
 6   Sgot Aspartate Aminotransferase       30229 non-null  float64
 7   Total Protiens                        30228 non-null  float64
 8    ALB Albumin                          30197 non-null  float64
 9   A/G Ratio Albumin and Globulin Ratio  30132 non-null  float64
 10  Result                                30691 non-null  int64  
dtypes: float64(9), 

In [ ]:
data.drop('Gender of the patient', axis=1, inplace=True)

In [ ]:
data = data.dropna()

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27666 entries, 0 to 30690
Data columns (total 10 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Age of the patient                    27666 non-null  float64
 1   Total Bilirubin                       27666 non-null  float64
 2   Direct Bilirubin                      27666 non-null  float64
 3    Alkphos Alkaline Phosphotase         27666 non-null  float64
 4    Sgpt Alamine Aminotransferase        27666 non-null  float64
 5   Sgot Aspartate Aminotransferase       27666 non-null  float64
 6   Total Protiens                        27666 non-null  float64
 7    ALB Albumin                          27666 non-null  float64
 8   A/G Ratio Albumin and Globulin Ratio  27666 non-null  float64
 9   Result                                27666 non-null  int64  
dtypes: float64(9), int64(1)
memory usage: 2.3 MB


In [ ]:
data.drop_duplicates()

Age of the patient  Total Bilirubin  Direct Bilirubin  \
0                    65.0              0.7               0.1   
1                    62.0             10.9               5.5   
2                    62.0              7.3               4.1   
3                    58.0              1.0               0.4   
4                    72.0              3.9               2.0   
...                   ...              ...               ...   
29894                32.0              0.6               0.1   
29895                61.0             10.2               4.2   
29974                58.0              2.9               1.3   
30688                54.0              6.8               3.0   
30689                48.0              1.9               1.0   

        Alkphos Alkaline Phosphotase   Sgpt Alamine Aminotransferase  \
0                              187.0                            16.0   
1                              699.0                            64.0   
2                              490.0                            60.0   
3                              182.0                            14.0   
4                              195.0                            27.0   
...                              ...                             ...   
29894                          196.0                            29.0   
29895                          232.0                            58.0   
29974                          482.0                            22.0   
30688                          542.0                           116.0   
30689                          231.0                            16.0   

       Sgot Aspartate Aminotransferase  Total Protiens   ALB Albumin  \
0                                 18.0             6.8           3.3   
1                                100.0             7.5           3.2   
2                                 68.0             7.0           3.3   
3                                 20.0             6.8           3.4   
4                                 59.0             7.3           2.4   
...                                ...             ...           ...   
29894                             30.0             5.8           2.9   
29895                            140.0             7.0           2.7   
29974                             34.0             7.0           2.4   
30688                             66.0             6.4           3.1   
30689                             55.0             4.3           1.6   

       A/G Ratio Albumin and Globulin Ratio  Result  
0                                      0.90       1  
1                                      0.74       1  
2                                      0.89       1  
3                                      1.00       1  
4                                      0.40       1  
...                                     ...     ...  
29894                                  1.00       1  
29895                                  0.60       1  
29974                                  0.50       1  
30688                                  0.90       1  
30689                                  0.60       1  

[14118 rows x 10 columns]

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27666 entries, 0 to 30690
Data columns (total 10 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Age of the patient                    27666 non-null  float64
 1   Total Bilirubin                       27666 non-null  float64
 2   Direct Bilirubin                      27666 non-null  float64
 3    Alkphos Alkaline Phosphotase         27666 non-null  float64
 4    Sgpt Alamine Aminotransferase        27666 non-null  float64
 5   Sgot Aspartate Aminotransferase       27666 non-null  float64
 6   Total Protiens                        27666 non-null  float64
 7    ALB Albumin                          27666 non-null  float64
 8   A/G Ratio Albumin and Globulin Ratio  27666 non-null  float64
 9   Result                                27666 non-null  int64  
dtypes: float64(9), int64(1)
memory usage: 2.3 MB


In [ ]:
result_mapping = { 1 : 0, 2:1}

In [ ]:
data['Result'] = data['Result'].map(result_mapping)

<ipython-input-57-185272255ac9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Result'] = data['Result'].map(result_mapping)


In [ ]:
X = data.drop('Result', axis=1)  # All columns except the target
y = data['Result']  # Target column

In [ ]:
y.isna().sum()

0

In [ ]:
gini_before = gini_impurity(y)
entropy_before = entropy(y)

In [ ]:
print(f"Gini Impurity before balancing: {gini_before}")
print(f"Entropy before balancing: {entropy_before}")

Gini Impurity before balancing: 0.4059324676956808
Entropy before balancing: 0.8596770667997339


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
feature_importances = pd.Series(rf.feature_importances_, index=X_train.columns).sort_values(ascending=False)
print("Feature Importances:\n", feature_importances)

Feature Importances:
  Alkphos Alkaline Phosphotase           0.174698
 Sgpt Alamine Aminotransferase          0.149223
Sgot Aspartate Aminotransferase         0.148245
Total Bilirubin                         0.132609
Total Protiens                          0.107845
 ALB Albumin                            0.107112
A/G Ratio Albumin and Globulin Ratio    0.092296
Direct Bilirubin                        0.087436
Age of the patient                      0.000537
dtype: float64


In [ ]:
class_counts = Counter(y)
min_class = min(class_counts, key=class_counts.get)

In [ ]:
df_minority = data[data['Result'] == min_class]
df_majority = data[data['Result'] != min_class]

In [ ]:
#randomly balanced data
df_majority_downsampled = df_majority.sample(len(df_minority))

In [ ]:
balanced_data = pd.concat([df_minority, df_majority_downsampled])

In [ ]:
y_balanced = balanced_data['Result']
gini_after = gini_impurity(y_balanced)
entropy_after = entropy(y_balanced)
print(f"Gini Impurity after balancing: {gini_after}")
print(f"Entropy after balancing: {entropy_after}")

Gini Impurity after balancing: 0.5
Entropy after balancing: 1.0


In [ ]:
X_balanced = balanced_data.drop('Result', axis=1)  # Replace 'Dataset' with your target column name
y_balanced = balanced_data['Result']
X_train_bal, X_test_bal, y_train_bal, y_test_bal = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=42)

In [ ]:
rf_bal = RandomForestClassifier(random_state=42)
rf_bal.fit(X_train_bal, y_train_bal)
y_pred_rf_bal = rf_bal.predict(X_test_bal)

In [ ]:
xgb = XGBClassifier(random_state=42)
xgb.fit(X_train_bal, y_train_bal)
y_pred_xgb = xgb.predict(X_test_bal)

In [ ]:
print("Random Forest Accuracy:", accuracy_score(y_test_bal, y_pred_rf_bal))
print("XGBoost Accuracy:", accuracy_score(y_test_bal, y_pred_xgb))
print("\nClassification Report for Random Forest:\n", classification_report(y_test_bal, y_pred_rf_bal))
print("\nClassification Report for XGBoost:\n", classification_report(y_test_bal, y_pred_xgb))

Random Forest Accuracy: 0.9990427568602425
XGBoost Accuracy: 0.9993618379068283

Classification Report for Random Forest:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      1569
           1       1.00      1.00      1.00      1565

    accuracy                           1.00      3134
   macro avg       1.00      1.00      1.00      3134
weighted avg       1.00      1.00      1.00      3134


Classification Report for XGBoost:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      1569
           1       1.00      1.00      1.00      1565

    accuracy                           1.00      3134
   macro avg       1.00      1.00      1.00      3134
weighted avg       1.00      1.00      1.00      3134



In [ ]:
y_test_bal

19633    0
25763    1
3077     0
5525     0
23285    1
        ..
23361    1
21774    0
20045    1
1218     0
24477    1
Name: Result, Length: 3134, dtype: int64

In [ ]:
y_pred_rf_bal

array([0, 1, 0, ..., 1, 0, 1])

In [ ]:
comparison_df = X_test_bal.copy()
comparison_df['Actual'] = y_test_bal
comparison_df['Predicted'] = y_pred_rf_bal

In [ ]:
wrong_predictions = comparison_df[comparison_df['Actual'] != comparison_df['Predicted']]

In [ ]:
wrong_predictions

Age of the patient  Total Bilirubin  Direct Bilirubin  \
29587                56.0              0.8               0.2   
9162                 45.0              2.2               1.0   
13532                51.0              1.9               1.0   

        Alkphos Alkaline Phosphotase   Sgpt Alamine Aminotransferase  \
29587                          158.0                            21.0   
9162                           610.0                            17.0   
13532                          231.0                            16.0   

       Sgot Aspartate Aminotransferase  Total Protiens   ALB Albumin  \
29587                             16.0             6.1           2.8   
9162                              28.0             7.3           2.6   
13532                             55.0             4.3           1.6   

       A/G Ratio Albumin and Globulin Ratio  Actual  Predicted  
29587                                  0.80       0          1  
9162                                   0.55       1          0  
13532                                  0.60       0          1

In [ ]:
comparison_df['Predicted'] = y_pred_xgb

In [ ]:
wrong_predictions = comparison_df[comparison_df['Actual'] != comparison_df['Predicted']]

In [ ]:
wrong_predictions

Age of the patient  Total Bilirubin  Direct Bilirubin  \
29587                56.0              0.8               0.2   
9162                 45.0              2.2               1.0   

        Alkphos Alkaline Phosphotase   Sgpt Alamine Aminotransferase  \
29587                          158.0                            21.0   
9162                           610.0                            17.0   

       Sgot Aspartate Aminotransferase  Total Protiens   ALB Albumin  \
29587                             16.0             6.1           2.8   
9162                              28.0             7.3           2.6   

       A/G Ratio Albumin and Globulin Ratio  Actual  Predicted  
29587                                  0.80       0          1  
9162                                   0.55       1          0